# Morphometric analysis

This notebook includes code to generate figures and statistical analyses for spinal cord morphometrics in pediatric subjects. 

In [26]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import yaml
import plotly.graph_objects as go
from statsmodels.stats.multitest import multipletests
from plotly.subplots import make_subplots
import statsmodels.formula.api as smf
import re

### Load config file to get path to dataset 

In [11]:
# Load config file
with open('../../config/config_preprocessing.yaml' , 'r') as file:
    config = yaml.safe_load(file)

# Get data path from config file
path_data = config['path_data']

### Get the `participants.tsv` file from the dataset

In [12]:
# Get path to participants.tsv file
participants_tsv = pd.read_csv(os.path.join(path_data, 'participants.tsv'), sep='\t')
participants_tsv

,participant_id,age,sex,group,scan_series,height,weight
0,sub-101,17,M,control,complete,1.778004,68.038864
1,sub-102,15,F,control,complete,1.625603,52.163129
2,sub-103,15,M,control,complete,1.651003,54.431091
3,sub-104,15,F,control,complete,1.625603,52.163129
4,sub-105,13,M,control,complete,1.524000,35.381000
...,...,...,...,...,...,...,...
110,sub-214,6,M,control,complete,NaN,NaN
111,sub-215,16,F,control,complete,NaN,NaN
112,sub-216,15,F,control,complete,NaN,NaN
113,sub-217,15,M,control,complete,NaN,NaN


## Dataframe of subjects included in the pipeline analysis

The following dataframe contains only the subjects that were included in this pipeline analysis.

In [13]:
def get_list_of_subjects_to_include(contrast, path_data, missing_data_subjects):
    """
    This function takes an image contrast (T2w, dwi, etc.), a path to a dataset, and a list of subjects with missing data,
    and returns a list of subjects to include in the analysis.

    The dataset needs to be in BIDS format, and the function will look for the participants.tsv file to get the list of subjects.
    The dataset should also contain an `exclude.yml` file that lists subjects to exclude from the analysis.
    """

    # Get the `participants.tsv` file and read it into a dataframe
    participants_tsv = pd.read_csv(os.path.join(path_data, 'participants.tsv'), sep='\t')

    # Get all subject IDs from the participants.tsv
    all_subjects = participants_tsv['participant_id'].tolist()

    # Get list of subjects to exclude from the analysis from the `exclude.yml` file
    with open(os.path.join(path_data, 'exclude.yml'), 'r') as file:
        exclude_yml = yaml.safe_load(file)

    exclude_t2star_key = exclude_yml.get(contrast, []) # Extract subjects under contrast key
    exclude_subjects = sorted(set(re.match(r"(sub-\d+)", entry).group(1) for entry in exclude_t2star_key if re.match(r"(sub-\d+)", entry))) # Extract the subject ID 

    # Add the list of subjects with missing data to the exclude_subjects list
    exclude_subjects.extend(missing_data_subjects)
    
    # Remove duplicates (if any), sort and print the list of subjects to exclude from the analysis
    exclude_subjects = sorted(set(exclude_subjects))
    print(f'subjects to exclude : {exclude_subjects}')

    # Compute the list of subjects to include in the analysis 
    include_subjects = [sub for sub in all_subjects if sub not in exclude_subjects]

    # Convert the list of included subjects to a dataframe
    include_subjects = participants_tsv[participants_tsv['participant_id'].isin(include_subjects)]

    return include_subjects

In [16]:
# List of subjects with missing dwi data
missing_t2_subjects = ["sub-217", 
                       "sub-107",
                       "sub-125",
                       "sub-137",
                       "sub-144",
                       "sub-152"]

# Get the list of subjects to include in the analysis
include_t2_subjects = get_list_of_subjects_to_include('t2w', path_data, missing_t2_subjects)
include_t2_subjects.to_csv(os.path.join('../tables/WMGM_distribution/include_t2star_subjects.csv'), sep='\t', index=False)

print(include_t2_subjects.shape[0], "subjects to include in the analysis")
print(f"\n list of subjects to include : \n {include_t2_subjects['participant_id'].tolist()}")

include_t2_subjects.head(20)

subjects to exclude : ['sub-107', 'sub-108', 'sub-110', 'sub-111', 'sub-121', 'sub-125', 'sub-133', 'sub-137', 'sub-141', 'sub-144', 'sub-150', 'sub-152', 'sub-159', 'sub-160', 'sub-161', 'sub-163', 'sub-165', 'sub-167', 'sub-168', 'sub-169', 'sub-170', 'sub-177', 'sub-179', 'sub-186', 'sub-190', 'sub-191', 'sub-193', 'sub-198', 'sub-203', 'sub-208', 'sub-209', 'sub-211', 'sub-213', 'sub-214', 'sub-217']
80 subjects to include in the analysis

 list of subjects to include : 
 ['sub-101', 'sub-102', 'sub-103', 'sub-104', 'sub-105', 'sub-106', 'sub-109', 'sub-112', 'sub-113', 'sub-114', 'sub-115', 'sub-116', 'sub-117', 'sub-118', 'sub-119', 'sub-120', 'sub-122', 'sub-123', 'sub-124', 'sub-126', 'sub-127', 'sub-128', 'sub-129', 'sub-130', 'sub-131', 'sub-132', 'sub-134', 'sub-135', 'sub-136', 'sub-138', 'sub-139', 'sub-140', 'sub-142', 'sub-143', 'sub-145', 'sub-146', 'sub-147', 'sub-148', 'sub-149', 'sub-151', 'sub-153', 'sub-154', 'sub-155', 'sub-156', 'sub-158', 'sub-162', 'sub-164', '

,participant_id,age,sex,group,scan_series,height,weight
0,sub-101,17,M,control,complete,1.778004,68.038864
1,sub-102,15,F,control,complete,1.625603,52.163129
2,sub-103,15,M,control,complete,1.651003,54.431091
3,sub-104,15,F,control,complete,1.625603,52.163129
4,sub-105,13,M,control,complete,1.524000,35.381000
5,sub-106,14,M,control,complete,1.701803,55.338276
8,sub-109,14,F,control,complete,1.498603,43.091281
11,sub-112,16,F,control,complete,1.676000,61.236000
12,sub-113,14,F,control,complete,1.676403,72.574788
13,sub-114,14,F,control,complete,1.574803,46.266428


## Plot demographics

This function plots the age and sex distribution of the subjects included in a pipeline analysis, according to the include list generated above. 

In [27]:
def plot_demographics(df):
    """
    This function plots the demographic information of participants, given a dataframe with the list of subjects to include in the analysis.
    """

    # Sort by sex
    df_M = df[df['sex'] == 'M']
    df_F = df[df['sex'] == 'F']

    # Round down age to nearest month 
    df['age'] = np.floor(df['age']) 

    # Create subplot
    fig = make_subplots(rows=1, cols=1)

    # Add histogram for female subjects
    fig.add_trace(go.Histogram(
        x=df_F['age'], 
        name='F', 
        marker=dict(color= "#D19D88"),
        opacity=1.0,
        legendgroup='F',
        ),
        row=1, col=1
    )

    # Add histogram for male subjects
    fig.add_trace(go.Histogram(
        x=df_M['age'], 
        name='M', 
        
        marker=dict(color="#5C8EA1"),
        opacity=1.0,
        legendgroup='M',
        ), 
        row=1, col=1
    )

    # Define age tick range
    tick_vals = list(range(6, 18)) 

    # Update layout
    fig.update_layout(
        width=900,
        height=500,
        font=dict(family='Arial', size=20, color='black'), 
        legend=dict(
            orientation="h", 
            yanchor="bottom", 
            y=1.0, 
            xanchor="center",  
            x=0.5,
        ),
        xaxis=dict(
            range=[5, 18],  # Set x-axis range from 6 to 17
        ),
        plot_bgcolor='white',
        barmode='stack',
        bargap=0.3,  
        xaxis_title='Age (years)',
        xaxis_title_font=dict(family='Arial', size=20, weight='bold'),
        yaxis_title='Number of Subjects',
        yaxis_title_font=dict(family='Arial', size=20, weight='bold'),
        xaxis_title_standoff=50, 
    )

    fig.update_xaxes(
        tickmode='array',
        tickvals=tick_vals,
        showgrid=False,
        gridwidth=1
    )

    fig.update_yaxes(
        showgrid=True,             # Horizontal grid lines
        gridcolor='lightgrey',
        gridwidth=1
    )

    # Set bin size to 1 year
    fig.update_traces(xbins=dict(size=1))

    fig.show()

In [28]:
# Plot demographics for included subjects in T2w analysis
plot_demographics(include_t2_subjects)

### Path to morphometrics

In [17]:
# Path to morphometric
morphometric_results_dir = '../tables/morphometrics/'

# Path to participants.tsv
participants_info_path = os.path.join(path_data, 'participants.tsv')

Load the CSV files containing morphometric values of each subject and add to a single dataframe

In [18]:
# Initialize an empty DataFrame to store morphometric values
morphometrics_df = pd.DataFrame()

# Load the CSV files containing morphometric values of each subject and add to a single dataframe
for file in os.listdir(morphometric_results_dir):
    if file.endswith('_final.csv'):
        df = pd.read_csv(os.path.join(morphometric_results_dir, file))
        morphometrics_df = pd.concat([morphometrics_df, df], ignore_index=True)

# Remove age and sex columns if they exist
if 'age' in morphometrics_df.columns:
    morphometrics_df = morphometrics_df.drop(columns=['age'])
if 'sex' in morphometrics_df.columns:
    morphometrics_df = morphometrics_df.drop(columns=['sex'])

In [19]:
morphometrics_df

""


In [20]:
# Add sex and age columns to the DataFrame (using participants.tsv)
participants_info = pd.read_csv(participants_info_path, sep='\t')
participants_info = participants_info.rename(columns={'participant_id': 'subject'})
participants_info

participants_info = participants_info[['subject', 'age', 'sex']]
morphometrics_df = morphometrics_df.merge(participants_info, on='subject', how='left')

KeyError: 'subject'

In [7]:
morphometrics_df

""


Function to generate a regression plot for a given morphometric (with respect to age) :

In [ ]:
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px
import webbrowser

def plot_morphometrics(df, morphometric_name_in_csv, morphometric_display_name):

    # Compute mean and std dev per age and vertebral level
    grouped = df.groupby(['age', 'VertLevel'])[morphometric_name_in_csv].agg(['mean', 'std']).reset_index()

    # Initialize figure
    fig = go.Figure()

    # Assign unique colors using Plotly's color sequence
    unique_ages = sorted(grouped['age'].unique())

    # Generate 12 evenly spaced colors from a sequential colormap
    colorscale = px.colors.diverging.Portland
    num_colors = 12
    color_gradient = px.colors.sample_colorscale(colorscale, [i / (num_colors - 1) for i in range(num_colors)])

    # Sort the ages and assign a color from the gradient
    unique_ages = sorted(grouped['age'].unique())
    color_map = {age: color_gradient[i] for i, age in enumerate(unique_ages)}

    # Plot each age group
    for age in unique_ages:
        group = grouped[grouped['age'] == age].sort_values('VertLevel')

        x = group['VertLevel']
        y_mean = group['mean']
        y_std = group['std']

        y_upper = y_mean + y_std
        y_lower = y_mean - y_std
        color = color_map[age]

        # # Add error band
        # fig.add_trace(go.Scatter(
        #     x=x.tolist() + x[::-1].tolist(),
        #     y=y_upper.tolist() + y_lower[::-1].tolist(),
        #     fill='toself',
        #     fillcolor=color.replace('rgb', 'rgba').replace(')', ',0.4)'),  # Make it transparent
        #     line=dict(color='rgba(255,255,255,0)'),
        #     hoverinfo='skip',
        #     showlegend=False,
        #     legendgroup=f'Age {age}',
        #     name=f'Age {age} ±1σ',
        #     opacity=0.1
        # ))

        # Add mean line
        fig.add_trace(go.Scatter(
            x=x,
            y=y_mean,
            mode='lines',
            name=f'{age}',
            line=dict(color=color, width=2),
            marker=dict(size=6),
            legendgroup=f'Age {age}'
        ))

    # Layout
    fig.update_layout(
        width=1000,
        height=600,
        title=f" ",
        xaxis_title='Vertebral Level',
        xaxis_title_font=dict(size=20, color='black', family='Arial', weight='bold'),
        yaxis_title=morphometric_display_name,
        yaxis_title_font=dict(size=20, color='black', family='Arial', weight='bold'),
        legend_title='Age',
        legend_title_font=dict(size=16, color='black', family='Arial', weight='bold'),
        legend_font=dict(size=14, color='black', family='Arial'),
        xaxis_tickangle=-45,
        template='plotly_white',
    )
    
    # Map VertLevel numbers to anatomical labels
    vertebral_label_map = {
        1.0: "C1", 2.0: "C2", 3.0: "C3", 4.0: "C4", 5.0: "C5",
        6.0: "C6", 7.0: "C7", 8.0: "T1", 9.0: "T2", 10.0: "T3",
        11.0: "T4", 12.0: "T5", 13.0: "T6", 14.0: "T7", 15.0: "T8",
        16.0: "T9", 17.0: "T10", 18.0: "T11", 19.0: "T12", 20.0: "L1"
    }

    # Extract all vertebral levels used in the data
    all_vert_levels = sorted(df['VertLevel'].dropna().unique())

    # Filter only integer levels (to show only C1, C2, etc.)
    integer_vert_levels = [v for v in all_vert_levels if v.is_integer() and v in vertebral_label_map]

    # Map to anatomical labels
    ticktexts = [vertebral_label_map[v] for v in integer_vert_levels]

    # Update the x-axis to show anatomical vertebral labels
    fig.update_xaxes(
        tickmode='array',
        tickvals=integer_vert_levels,
        ticktext=ticktexts,
        tickcolor='black',
        tickfont=dict(size=16, color='black', family='Arial'),
        tickangle=0,
        gridcolor='rgba(0, 0, 0, 0.1)',  
        
    )
    
    fig.update_yaxes(
        gridcolor='rgba(0, 0, 0, 0.1)',
        tickcolor='black',
        tickfont=dict(size=16, color='black', family='Arial')
    )

    # Define the save path
    save_path = os.path.join('..', 'figures', f'{morphometric_name_in_csv}_vs_Vert_Levels.html')

    # Save the figure to an HTML file
    fig.write_html(save_path)

    # Open the file in the default web browser
    webbrowser.open(f'file://{os.path.abspath(save_path)}')

    # Show the figure in the notebook as well
    fig.show()

### 1. Spinal cord cross-sectional area (CSA)

In [ ]:
plot_morphometrics(morphometrics_df, 'CSA','CSA (mm²)')

### 2. Antero-posterior (AP) diameter, right-left (RL) diameter

In [ ]:
plot_morphometrics(morphometrics_df, 'AP_diameter', 'AP diameter (mm)')
plot_morphometrics(morphometrics_df, 'RL_diameter', 'RL diameter (mm)')

### 3. Eccentricity and solidity

In [ ]:
plot_morphometrics(morphometrics_df, 'eccentricity', 'Eccentricity (mm)')

### 5. Spinal levels vs vertebral levels